# Document Clustering

importing needed libraries and data

In [1]:
%load_ext autoreload
%autoreload 2

import json
from collections import defaultdict, Counter


#General-Purpose
import pandas as pd
import numpy as np
import re
import nltk
import pickle
import shutil
import os
from tqdm import tqdm

#Preprocessing
#from utils.pipeline_v2b import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD

#Vectorization
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Clustering
from sklearn.cluster import KMeans, HDBSCAN, OPTICS

#Evaluation
from sklearn.metrics import silhouette_score, calinski_harabasz_score, adjusted_mutual_info_score

#Visualization
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

#pandas and numpy for df manipulation
import pandas as pd
import re
import nltk
nltk.download('averaged_perceptron_tagger_eng')

#Preprocessing: tokenization
from nltk.tokenize import RegexpTokenizer

#NER and Metrics with sklearn and sklearn_crfsuite
from sklearn.model_selection import train_test_split
#%pip install git+"https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite"
import sklearn_crfsuite
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers

#NER and Visualization with SpaCy
#%pip install -U pip setuptools wheel
#%pip install -U spacy
#!python -m spacy download en_core_web_sm
import spacy
from spacy import displacy

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
reviews = pd.read_csv(r"D:\MARGARIDA\dificuldade\3rd_year\TM project\Hyderabadi-Word-Soup\coocurrence_clustering\reviews_with_dishes.csv")

## Data Preparation

Preparing data using the best performing model for NER detected in co_occurance_exploration

vectorizing

In [3]:
def data_preparer(data, column, sample_size=0.25):
    """
    Prepares data for NLP tasks by vectorizing text data.
    
    Args:
        data (DataFrame): Input data.
        column (str): Name of the column containing text data.
        sample_size (float, optional): Proportion of data to use for sampling. Defaults to 0.25.
    
    Returns:
        DataFrame: Data with vectorized text data.
    """
    # Vectorize using BOW
    bow_vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r"(?u)\b\w+\b")
    data_bow_td_matrix = bow_vectorizer.fit_transform(data[column].apply(lambda x: ' '.join(x))).toarray()
    data["bow_vector"] = data_bow_td_matrix.tolist()

    # Vectorize using TF-IDF
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), token_pattern=r"(?u)\b\w+\b")
    data_tfidf_td_matrix = tfidf_vectorizer.fit_transform(data[column].apply(lambda x: ' '.join(x))).toarray()
    data["tfidf_vector"] = data_tfidf_td_matrix.tolist()

    # Vectorize using Doc2Vec
    d2v = Doc2Vec 
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data[column])]
    d2v_model = d2v(documents, vector_size=300, window=6, min_count=1, workers=4, epochs=20)
    data["doc2vec_vector"] = [d2v_model.dv[idx].tolist() for idx in tqdm(range(len(data)))]

    return data

In [4]:
vectorized_reviews = data_preparer(reviews, "filtered_review")

100%|██████████| 9955/9955 [00:00<00:00, 28990.84it/s]


In [5]:
ner_bow_td_matrix= np.array([[component for component in doc] for doc in vectorized_reviews["bow_vector"]])
ner_tfidf_td_matrix = np.array([[component for component in doc] for doc in vectorized_reviews["tfidf_vector"]])
ner_doc2vec_td_matrix = np.array([[component for component in doc] for doc in vectorized_reviews["doc2vec_vector"]])

In [6]:
kmeans_tfidf = KMeans(n_clusters=9, random_state=39).fit(ner_tfidf_td_matrix)